In [1]:
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier
import os
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")


/home/idio/anaconda3/lib/python3.12/site-packages/speechbrain/utils/autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


In [ ]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="",
    api_key="",
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='celebrity_voices')]


In [8]:
collection_name = "celebrity_voices"

In [5]:
from qdrant_client.models import VectorParams, Distance

collection_name = "celebrity_voices"

qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=192,
        distance=Distance.COSINE,
        hnsw_config={
            "m": 16,
            "ef_construct": 100,
        }
    )
)


/tmp/ipykernel_36781/755363720.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [3]:
base_dir = 'dataset/vox1_test_wav/wav'
embeddings = []
metadata_list = []

# Outer loop for each speaker id directory (e.g., 'id10271', 'id10272', ...)
for speaker_id in os.listdir(base_dir):
    speaker_path = os.path.join(base_dir, speaker_id)
    if os.path.isdir(speaker_path):
        # Inner loop for each celebrity subfolder (e.g., '1gtz-CUIygI')
        for celeb_folder in os.listdir(speaker_path):
            folder_path = os.path.join(speaker_path, celeb_folder)
            if os.path.isdir(folder_path):
                # Loop through each WAV file in the celeb's folder
                for file_name in os.listdir(folder_path):
                    if file_name.endswith('.wav'):
                        file_path = os.path.join(folder_path, file_name)
                        signal, fs = torchaudio.load(file_path)
                        embedding = classifier.encode_batch(signal).flatten()
                        embeddings.append(embedding)
                        # Remove file extension to get number
                        num = os.path.splitext(file_name)[0]
                        metadata = {'id': speaker_id, 'path': celeb_folder, 'num': num}
                        metadata_list.append(metadata)

print("Total embeddings created:", len(embeddings))
print("Example metadata:", metadata_list[0])

Total embeddings created: 4874
Example metadata: {'id': 'id10271', 'path': 'OZcAiOZBNjQ', 'num': '00001'}


In [6]:
ids = [i for i in range(len(embeddings))]

In [9]:
batch_size = 1000

for i in range(0, len(embeddings), batch_size):
    batch_points = [
        {
            "id": ids[j],
            "vector": embeddings[j],
            "payload": metadata_list[j]
        }
        for j in range(i, min(i + batch_size, len(embeddings)))
    ]

    qdrant_client.upsert(
        collection_name=collection_name,
        points=batch_points
    )


In [10]:
all_points = []
next_page_offset = None

while True:
    points, next_page_offset = qdrant_client.scroll(
        collection_name=collection_name,
        offset=next_page_offset,
        limit=100,
        with_payload=True,
        with_vectors=True
    )

    all_points.extend(points)

    if next_page_offset is None:
        break